In [2]:

import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import datasets, layers, models


# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)



2022-11-02 15:05:25.633051: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-02 15:05:27.248507: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdirectml.d6f03b303ac3c4f2eeb8ca631688c9757b361310.so
2022-11-02 15:05:27.248571: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdxcore.so
2022-11-02 15:05:27.260047: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libd3d12.so
2022-11-02 15:05:29.214519: I tensorflow/c/logging.cc:34] DirectML device enumeration: found 1 compatible adapters.


## Modelo de identificación de placas

In [6]:
model = models.Sequential([

#   Convolution layers
    layers.Conv2D(64, (3, 3), activation='relu', padding="same", input_shape=(256, 256, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu', padding="same"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (3, 3), activation='relu', padding="same"),

#   Normal layers
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dropout(0.5), # Evita la conexión total entre capaz para evitar overfitting
    layers.Dense(32,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(32,activation='relu'),
    layers.Dense(5,activation='relu'),   # 5 neuronas de salida, desempeño + fronteras
])
model.summary()

model.compile(optimizer="Adam",
              loss="categorical_crossentropy",
              metrics=['accuracy'])

# Comentarios

# 1. muestreo %validación %entrenamiento
# 2. Cargar imágenes
# 3. Modelo
# 4. Entrenamiento
# 5. Validación
# 6. gráficas entrenamiento validación
# 7. Función de aplicación 

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 256, 256, 64)      1792      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 128, 128, 64)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 128, 128, 32)      18464     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 64, 64, 16)        4624      
                                                                 
 flatten_1 (Flatten)         (None, 65536)            

## Carga de Datos

In [11]:
# Load csv
plates_data = pd.read_csv("./normalized_data/unique_plates.csv")

# Take a sample of the data
number_of_samples = 500;
plates_data_sample = plates_data.sample(number_of_samples)

# Create arrays with inputs and outputs
plates_data_in = np.ndarray(shape=(number_of_samples, 256, 256, 3), dtype=np.uint8)
plates_data_out = np.array([plates_data_sample['tag'],
                            plates_data_sample['xmin'],
                            plates_data_sample['ymin'],
                            plates_data_sample['xmax'],
                            plates_data_sample['ymax']]).T


# Load images from paths in csv

for i, path in enumerate(plates_data_sample['img_path']):
    raw_image = cv2.imread(path, cv2.IMREAD_COLOR)
    plates_data_in[i,:,:,:] = cv2.cvtColor(raw_image, cv2.COLOR_BGR2RGB)  # Reorganize channels to RGB instead of BGR

# print first column of plates_data_sample
print(plates_data_sample['img_path'])
#plt.imshow(plates_data_in[i])
#plt.show()

3099    /home/camilo/repos/TIA_2022/normalized_data/pl...
1381    /home/camilo/repos/TIA_2022/normalized_data/pl...
7793    /home/camilo/repos/TIA_2022/normalized_data/pl...
8913    /home/camilo/repos/TIA_2022/normalized_data/pl...
1757    /home/camilo/repos/TIA_2022/normalized_data/pl...
                              ...                        
5701    /home/camilo/repos/TIA_2022/normalized_data/pl...
228     /home/camilo/repos/TIA_2022/normalized_data/pl...
43      /home/camilo/repos/TIA_2022/normalized_data/pl...
752     /home/camilo/repos/TIA_2022/normalized_data/pl...
2470    /home/camilo/repos/TIA_2022/normalized_data/pl...
Name: img_path, Length: 500, dtype: object


## Entrenamiento

In [5]:
history = model.fit(plates_data_in, plates_data_out,
                    validation_split=0.2,
                    epochs=10,
                    batch_size = 10).history

# model.save('models/model.h5')

ValueError: Failed to find data adapter that can handle input: <class 'NoneType'>, <class 'NoneType'>